In [9]:
import pandas as pd
import numpy as np
import os, sys
import math

In [10]:
def apply_pbc(value):
    if abs(value) > 0.5:
        return 1 - abs(value)
    return value


def mic_eucledian_distance(coor1, coor2):
    x_coor1, y_coor1, z_coor1 = coor1
    x_coor2, y_coor2, z_coor2 = coor2
    
    delta_x = x_coor1 - x_coor2
    delta_y = y_coor1 - y_coor2
    delta_z = z_coor1 - z_coor2

    distance = math.sqrt(sum([(apply_pbc(delta_x))**2, (apply_pbc(delta_y))**2, (apply_pbc(delta_z))**2]))
    return distance


def get_duplicate_coor_final_closest_w_data(dict):
    duplicate_coor_final_closest = {}
    for i, values in dict.items():
        for entry in values:
            coor_init = entry["coor_init"]
            coor_final_closest = entry["coor_final_closest"]
            dist = entry["dist"]

        if coor_final_closest in duplicate_coor_final_closest:
            duplicate_coor_final_closest[coor_final_closest].append({"coor_init": coor_init, "dist": dist})
        else:
            duplicate_coor_final_closest[coor_final_closest] = [{"coor_init": coor_init, "dist": dist}]

    duplicate_coor_final_closest_w_data = {}
    for coor_final_closest, coor_inits_dists in duplicate_coor_final_closest.items():
        if len(coor_inits_dists) > 1:
            duplicate_coor_final_closest_w_data[coor_final_closest] = [{"coor_inits and dists": coor_inits_dists}]

    return duplicate_coor_final_closest_w_data

# def get_coordinate_POSCAR(file_path):
#     with open(file_path, 'r') as file:
#         lines = file.readlines()
#     data = lines[poscar_line_nr_start:poscar_line_nr_start+amount_Li]

#     df = pd.DataFrame([string.strip().split() for string in data])
#     df_float = df.astype(float)

#     check_negative = df_float.lt(0).any().any()

#     if check_negative == True:
#         breaksys.exit()    

#     coordinate = [row.to_numpy() for index, row in df_float.iterrows()]

#     return coordinate

In [11]:
# class PoscarReader:
#     def __init__(self, file_path):
#         self.file_path = file_path
#         self.poscar_line_nr_start = 8   # index from 0
#         self.poscar_line_nr_end = 60
#         self.amount_Li = 24
#         self.n_decimal = 16
        

#     def get_coordinate(self):
#         with open(self.file_path, 'r') as file:
#             lines = file.readlines()

#         data = lines[self.poscar_line_nr_start : self.poscar_line_nr_start+self.amount_Li]
#         data_non_Li = lines[self.poscar_line_nr_start+self.amount_Li : self.poscar_line_nr_end]

#         df = pd.DataFrame([string.strip().split() for string in data])
#         df_float = df.astype(float)

#         df_non_Li = pd.DataFrame([string.strip().split() for string in data_non_Li])
#         df_float_non_Li = df_non_Li.astype(float)

#         check_negative = df_float.lt(0).any().any()

#         if check_negative:
#             sys.exit()

#         coordinates = [row.to_numpy() for index, row in df_float.iterrows()]

#         return coordinates

In [12]:
file_path_init = "0_2_CONTCAR"
file_path = "6_6_CONTCAR"

In [13]:
# poscar_reader_init = PoscarReader(file_path_init)
# coordinate_init = poscar_reader_init.get_coordinate()

# poscar_reader = PoscarReader(file_path)
# coordinate = poscar_reader.get_coordinate()

In [14]:
poscar_line_nr_start = 8        # index from 0
poscar_line_nr_end = 60
amount_Li = 24
n_decimal = 16

In [15]:
with open(file_path_init, 'r') as file_init:
    lines_init = file_init.readlines()
data_init = lines_init[poscar_line_nr_start:poscar_line_nr_start+amount_Li]

df_init = pd.DataFrame([string.strip().split() for string in data_init])
df_float_init = df_init.astype(float)

check_negative_init = df_float_init.lt(0).any().any()

if check_negative_init == True:
    breaksys.exit()    

coordinate_init = [row.to_numpy() for index, row in df_float_init.iterrows()]

In [16]:
with open(file_path, 'r') as file:
    lines = file.readlines()
data = lines[poscar_line_nr_start:poscar_line_nr_start+amount_Li]
data_non_Li = lines[poscar_line_nr_start+amount_Li:poscar_line_nr_end]

df = pd.DataFrame([string.strip().split() for string in data])
df_float = df.astype(float)
df_non_Li = pd.DataFrame([string.strip().split() for string in data_non_Li])
df_float_non_Li = df_non_Li.astype(float)

check_negative = df_float.lt(0).any().any()

if check_negative == True:
    breaksys.exit()

coordinate = [row.to_numpy() for index, row in df_float.iterrows()]

In [17]:
coor_final_reindex_dict_value = {}
coor_final_reindex_dict = {}

for i, coor_init in enumerate(coordinate_init):
    coor_final_reindex_dict_value['coor_init'] = tuple(coor_init)
    coor_final_reindex_dict_value['coor_final_closest'] = None
    coor_final_reindex_dict_value['dist'] = None

    coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]


def get_coor_final_reindex_dict(coor_final_reindex_dict, coordinate):
    # coor_final_reindex = []
    # coor_final_reindex_dict = {}

    for i, value in coor_final_reindex_dict.items():
        distance_prev = float("inf")
        coor_final_closest = None

        for entry in value:
            coor_init = entry['coor_init']

            for j, coor in enumerate(coordinate):
                distance = mic_eucledian_distance(coor_init, coor)

                if distance < distance_prev:
                    distance_prev = distance
                    coor_final_closest = coor

        # coor_final_reindex.append(coor_final_closest)

            if entry['coor_final_closest'] == None and entry['dist'] == None:
                entry['coor_final_closest'] = tuple(coor_final_closest)
                entry['dist'] = distance_prev

        # if i in coor_final_reindex_dict:
        # new_entry = coor_final_reindex_dict[i].copy()
        # new_entry.append(coor_final_reindex_dict_value)
        # coor_final_reindex_dict[i] = new_entry
        # else:
        #     coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]

    return coor_final_reindex_dict


def get_coor_final_reindex_dict_eliminated(coor_final_reindex_dict, duplicate_coor_final_w_data):
    for key, value in duplicate_coor_final_w_data.items():
        # Extract the list of coor_inits and dists
        coor_inits_and_dists = value[0]['coor_inits and dists']
        
        # # Find the coor_init with the biggest distance, and then delete it from coordinate
        max_distance_coor_init = max(coor_inits_and_dists, key=lambda x: x['dist'])['coor_init']

        # print("Duplicate coor_final_closest:", key)
        # # for i in coor_inits_and_dists:
        # #     print(i['coor_init'])
        # print("coor_init with the biggest distance:", max_distance_coor_init)
        # # print()

        coor_final_reindex_dict_eliminated = coor_final_reindex_dict.copy()

        # Find the corresponding entry and delete it
        for idx, values in coor_final_reindex_dict_eliminated.items():
            for entry in values:
                if entry['coor_init'] == max_distance_coor_init and entry['coor_final_closest'] == key:
                    entry['coor_final_closest'] = None
                    entry['dist'] = None
                    # del entry[0]['coor_final_closest']
                    # del entry[0]['dist']
                    # # entry['coor_init'] = max_distance_coor_init

    return coor_final_reindex_dict_eliminated


def get_coor_final_closest_eliminated(coor_final_reindex_dict_eliminated, coordinate):
    # coor_init_array = []
    coor_final_closest_eliminated_array = coordinate.copy()

    for key, values in coor_final_reindex_dict_eliminated.items():
        for entry in values:
            if np.all(entry['coor_final_closest'] == coordinate):
                coor_final_closest_eliminated_array.remove(entry['coor_final_closest'])

    return coor_final_closest_eliminated_array


    # # coor_init_array = np.array(coor_init_array)
    # coor_final_closest_eliminated_array = np.array(coor_final_closest_eliminated_array)


    # idx_empty = []

    # for idx, values in coor_final_reindex_dict_eliminated.items():
    #     for entry in values:
    #         if entry['dist'] == None:
    #             idx_empty.append(idx)
    #             print(idx)

In [18]:
# # coor_final_reindex_dict, duplicate_coor_f    
# coor_final_reindex_dict_value = {}
# coor_final_reindex_dict = {}

# for i, coor_init in enumerate(coordinate_init):
#     distance_prev = float("inf")
#     closest24 = None

#     for j, coor in enumerate(coordinate):
#         distance = mic_eucledian_distance(coor_init, coor)

#         if distance < distance_prev:
#             distance_prev = distance
#             coor_final_closest = coor

#     # coor_final_reindex.append(coor_final_closest)

#     coor_final_reindex_dict_value['coor_init'] = tuple(coor_init)
#     coor_final_reindex_dict_value['coor_final_closest'] = tuple(coor_final_closest)
#     coor_final_reindex_dict_value['dist'] = distance_prev

#     if i in coor_final_reindex_dict:
#         new_entry = coor_final_reindex_dict[i].copy()
#         new_entry.append(coor_final_reindex_dict_value)
#         coor_final_reindex_dict[i] = new_entry
#     else:
#         coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]

#     duplicate_coor_final_w_data = get_duplicate_coor_final_closest_w_data(coor_final_reindex_dict)

In [19]:
# # coor_final_reindex_dict_value = {}
# # coor_final_reindex_dict = {}

# # for i, coor_init in enumerate(coordinate_init):
# #     coor_final_reindex_dict_value['coor_init'] = tuple(coor_init)

# #     coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]

In [20]:
# coor_final_reindex_dict

In [21]:
# # # coor_final_reindex = []
# # coor_final_reindex_dict_value = {}
# # # coor_final_reindex_dict = {}

# # for i, value in coor_final_reindex_dict.items():
# #     distance_prev = float("inf")
# #     closest24 = None

# #     for entry in value:
# #         coor_init = entry['coor_init']

# #         for j, coor in enumerate(coordinate):
# #             distance = mic_eucledian_distance(coor_init, coor)

# #             if distance < distance_prev:
# #                 distance_prev = distance
# #                 coor_final_closest = coor

# #     # coor_final_reindex.append(coor_final_closest)

# #         entry['coor_final_closest'] = tuple(coor_final_closest)
# #         entry['dist'] = distance_prev

# #     # if i in coor_final_reindex_dict:
# #     # new_entry = coor_final_reindex_dict[i].copy()
# #     # new_entry.append(coor_final_reindex_dict_value)
# #     # coor_final_reindex_dict[i] = new_entry
# #     # else:
# #     #     coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]

# # duplicate_coor_final_w_data = get_duplicate_coor_final_closest_w_data(coor_final_reindex_dict)
        
# #     # if tuple(coor_init) in coor_final_reindex_dict:
# #     #     new_entry = coor_final_reindex_dict[tuple(coor_init)].copy()
# #     #     new_entry.append(coor_final_reindex_dict_value)
# #     #     coor_final_reindex_dict[tuple(coor_init)] = new_entry
# #     # else:
# #     #     coor_final_reindex_dict[tuple(coor_init)] = [coor_final_reindex_dict_value.copy()]

In [22]:
# get init dict
coor_final_reindex_dict = get_coor_final_reindex_dict(coor_final_reindex_dict, coordinate)
# get duplicate
duplicate_coor_final_w_data = get_duplicate_coor_final_closest_w_data(coor_final_reindex_dict)
# works only if duplicate 1 atm
if len(duplicate_coor_final_w_data) > 1:
    # Create array from 'coor_init' and 'coor_final_closest'
    # # coor_init_array = np.array([entry[0]['coor_init'] for entry in coor_final_reindex_dict.values()])
    coor_final_closest_array = np.array([entry[0]['coor_final_closest'] for entry in coor_final_reindex_dict.values()])
    print(coor_final_closest_array == coordinate)
    print("Please check manually")
    sys.exit()
elif len(duplicate_coor_final_w_data) == 1:
    # # get dict eliminated
    # coor_final_reindex_dict_eliminated = get_coor_final_reindex_dict_eliminated(coor_final_reindex_dict, duplicate_coor_final_w_data)
    for key, value in duplicate_coor_final_w_data.items():
        # Extract the list of coor_inits and dists
        coor_inits_and_dists = value[0]['coor_inits and dists']
        
        # # Find the coor_init with the biggest distance, and then delete it from coordinate
        max_distance_coor_init = max(coor_inits_and_dists, key=lambda x: x['dist'])['coor_init']

        # print("Duplicate coor_final_closest:", key)
        # # for i in coor_inits_and_dists:
        # #     print(i['coor_init'])
        # print("coor_init with the biggest distance:", max_distance_coor_init)
        # # print()

        coor_final_reindex_dict_eliminated = coor_final_reindex_dict.copy()

        # Find the corresponding entry and delete it
        for idx, values in coor_final_reindex_dict_eliminated.items():
            for entry in values:
                if entry['coor_init'] == max_distance_coor_init and entry['coor_final_closest'] == key:
                    entry['coor_final_closest'] = None
                    entry['dist'] = None
                    # del entry[0]['coor_final_closest']
                    # del entry[0]['dist']
                    # # entry['coor_init'] = max_distance_coor_init

    coor_final_closest_eliminated_array = coordinate.copy()

    for key, values in coor_final_reindex_dict_eliminated.items():
        for entry in values:
            for i in coordinate:
                if (entry['coor_final_closest'] == i).any():
                    # coor_final_closest_eliminated_array.remove(entry['coor_final_closest'])
                    coor_final_closest_eliminated_array = [arr for arr in coor_final_closest_eliminated_array if not np.array_equal(arr, i)]
    
    # # get coor eliminated
    # coor_final_closest_eliminated_array = get_coor_final_closest_eliminated(coor_final_reindex_dict_eliminated, coordinate)

    # repeat
    coor_final_reindex_dict_eliminated_final = get_coor_final_reindex_dict(coor_final_reindex_dict_eliminated, coor_final_closest_eliminated_array)

elif len(duplicate_coor_final_w_data) == 0:
    coor_final_reindex_dict_eliminated_final = coor_final_reindex_dict

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [False False False]
 [ True  True  True]
 [False False False]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [False False False]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]]
Please check manually


SystemExit: 

/home/azka/anaconda3/envs/positionism/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
# Create array from 'coor_init'
coor_init_array = np.array([entry[0]['coor_init'] for entry in coor_final_reindex_dict.values()])

def safe_none(val):
    return val if val is not None else (np.nan, np.nan, np.nan)

# Create array from 'coor_final_closest'
coor_final_closest_array = np.array([safe_none(entry[0]['coor_final_closest']) for entry in coor_final_reindex_dict.values()])
# coor_final_closest_array = np.array([entry[0]['coor_final_closest'] for entry in coor_final_reindex_dict.values()])

In [24]:
coor_final_closest_array == coordinate

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [False, False, False],
       [ True,  True,  True],
       [False, False, False],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [False, False, False],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [34]:
print(coordinate[4])
print(coordinate[6])
print(coordinate[17])

[0.85322758 0.97758915 0.78709747]
[0.69156027 0.29595077 0.00545842]
[0.15016612 0.54587657 0.32494944]


In [35]:
print(coor_final_closest_array[4])
print(coor_final_closest_array[6])
print(coor_final_closest_array[17])

[0.19770637 0.04575638 0.82333474]
[0.96097194 0.18505776 0.15257308]
[0.45870625 0.67955933 0.19140088]


In [36]:
coor_final_closest_array

array([[0.96097194, 0.18505776, 0.15257308],
       [0.00294494, 0.6959242 , 0.71203714],
       [0.05756533, 0.33907023, 0.69803035],
       [0.02218655, 0.8385717 , 0.21720905],
       [0.19770637, 0.04575638, 0.82333474],
       [0.21164387, 0.7953013 , 0.01784438],
       [0.96097194, 0.18505776, 0.15257308],
       [0.80486511, 0.79832725, 0.98995602],
       [0.31390979, 0.9884715 , 0.27773931],
       [0.19770637, 0.04575638, 0.82333474],
       [0.55407283, 0.84751941, 0.64850414],
       [0.6696837 , 0.03980916, 0.32222791],
       [0.29647946, 0.79998377, 0.48506139],
       [0.28223325, 0.21188157, 0.51173593],
       [0.80716316, 0.69767181, 0.51682454],
       [0.81762007, 0.32788652, 0.45408675],
       [0.30538804, 0.37291854, 0.94292993],
       [0.45870625, 0.67955933, 0.19140088],
       [0.71492452, 0.49244209, 0.21100062],
       [0.77760336, 0.51084633, 0.73962249],
       [0.49040193, 0.18581378, 0.69500394],
       [0.45870625, 0.67955933, 0.19140088],
       [0.

In [28]:
print(mic_eucledian_distance(coordinate_init[4], coordinate[4]))
print(mic_eucledian_distance(coordinate_init[4], coordinate[6]))
print(mic_eucledian_distance(coordinate_init[4], coordinate[17]))


0.414757036420287
0.5075731096498405
0.5282488306418942


In [29]:
print(mic_eucledian_distance(coordinate_init[6], coordinate[4]))
print(mic_eucledian_distance(coordinate_init[6], coordinate[6]))
print(mic_eucledian_distance(coordinate_init[6], coordinate[17]))


0.3124989414982152
0.19623725129315142
0.5770970020976907


In [30]:
print(mic_eucledian_distance(coordinate_init[17], coordinate[4]))
print(mic_eucledian_distance(coordinate_init[17], coordinate[6]))
print(mic_eucledian_distance(coordinate_init[17], coordinate[17]))


0.7452138272374241
0.4634744384637823
0.24438465806179663


In [ ]:
duplicate_coor_final_w_data

In [ ]:
# coor_final_closest_eliminated_array

In [ ]:
# duplicate_coor_final_w_data

In [ ]:
# coor_final_reindex_dict

In [ ]:
# coor_final_reindex_dict_eliminated

In [ ]:
# coor_final_reindex_dict_eliminated_final

In [ ]:
# duplicate_coor_final_w_data = get_duplicate_coor_final_closest_w_data(coor_final_reindex_dict)

In [ ]:
# for key, value in duplicate_coor_final_w_data.items():
#     # Extract the list of coor_inits and dists
#     coor_inits_and_dists = value[0]['coor_inits and dists']
    
#     # # Find the coor_init with the biggest distance, and then delete it from coordinate
#     max_distance_coor_init = max(coor_inits_and_dists, key=lambda x: x['dist'])['coor_init']

#     # print("Duplicate coor_final_closest:", key)
#     # # for i in coor_inits_and_dists:
#     # #     print(i['coor_init'])
#     # print("coor_init with the biggest distance:", max_distance_coor_init)
#     # # print()

#     coor_final_reindex_dict_eliminated = coor_final_reindex_dict.copy()

#     # Find the corresponding entry and delete it
#     for idx, values in coor_final_reindex_dict_eliminated.items():
#         for entry in values:
#             if entry['coor_init'] == max_distance_coor_init and entry['coor_final_closest'] == key:
#                 entry['coor_final_closest'] = None
#                 entry['dist'] = None
#                 # del entry[0]['coor_final_closest']
#                 # del entry[0]['dist']
#                 # # entry['coor_init'] = max_distance_coor_init



In [ ]:
# coor_final_reindex_dict_eliminated

In [ ]:
# coor_final_reindex_dict_eliminated[0][0]['coor_init']

In [ ]:
# coor_final_reindex_dict_eliminated

In [ ]:
# # coor_init_array = []
# coor_final_closest_eliminated_array = coor_final_closest.copy()

# for key, values in coor_final_reindex_dict_eliminated.items():
#     for entry in values:
#         if np.all(entry['coor_final_closest'] == coordinate):
#             coor_final_closest_eliminated_array.remove(entry['coor_final_closest'])


# # # coor_init_array = np.array(coor_init_array)
# # coor_final_closest_eliminated_array = np.array(coor_final_closest_eliminated_array)

In [ ]:
# coor_final_closest_eliminated_array

In [ ]:
# idx_empty = []

# for idx, values in coor_final_reindex_dict_eliminated.items():
#     for entry in values:
#         if entry['dist'] == None:
#             idx_empty.append(idx)
#             print(idx)

In [ ]:
# coordinate

In [ ]:
# coor_final_closest_array = []

# for i in idx_empty:
#     coor_final_closest_array.append(coordinate_init[i])

#     for idx, values in coor_final_reindex_dict_eliminated.items():
#         if i == idx:
#             coor_final_reindex_dict_eliminated[i][0]['coor_init'] = coordinate_init[i]

In [ ]:
coordinate_final = []

for key, values in coor_final_reindex_dict_eliminated_final.items():
    for entry in values:
        coor_final_closest = entry['coor_final_closest']

        coordinate_final.append(coor_final_closest)

In [ ]:
# duplicate_coor_final_w_data

In [ ]:
# # Find duplicate rows and get their indices
# unique_rows, indices, counts = np.unique(coor_final_reindex, axis=0, return_index=True, return_counts=True)

# # Get the indices of duplicate rows
# duplicate_indices = indices[counts > 1]

# if len(duplicate_indices) > 0:
#     print("Duplicate rows found at indices:", duplicate_indices)
# else:
#     print("No duplicate rows found.")

In [ ]:
# counts

In [ ]:
# indices

In [ ]:
are_equal = all(np.array_equal(coor_reindex, coor) for coor_reindex, coor in zip(coordinate_final, coordinate))

if are_equal:
    print("The arrays are the same.")
else:
    print("The arrays are different.")
    indices_diff = [i for i, (reindex, coord) in enumerate(zip(coordinate_final, coordinate)) if not np.array_equal(reindex, coord)]
    print("Indices where the arrays differ:", indices_diff)

In [ ]:
# df_coordinate = pd.DataFrame(coordinate)
# df_concat = pd.concat([df_coordinate, df_float_non_Li], ignore_index=True)

In [ ]:
# for i in range(df_concat.shape[0]):
#     for j in range(df_concat.shape[1]):
#         temp = df_concat[j][i]
#         df_concat[j][i] = '{:.{width}f}'.format(float(temp), width=n_decimal)

In [ ]:
df_coordinate = pd.DataFrame(coordinate_final)
df_concat = pd.concat([df_coordinate, df_float_non_Li], ignore_index=True)

for i in range(df_concat.shape[0]):
    for j in range(df_concat.shape[1]):
        temp = df_concat[j][i]
        df_concat[j][i] = '{:.{width}f}'.format(float(temp), width=n_decimal)

row_list = df_concat.to_string(index=False, header=False).split('\n')
row_list_space = ['  '.join(string.split()) for string in row_list] # 2 spaces of distance
row_list_w_beginning = ['  ' + row for row in row_list_space]       # 2 spaces in the beginning
absolute_correct_list = '\n'.join(row_list_w_beginning).splitlines()        

line_append_list = []
for idx_c, line in enumerate(absolute_correct_list):
    line_new_line = str(line) + '\n'
    line_append_list.append(line_new_line)

file_list = lines[:poscar_line_nr_start] + line_append_list

poscar_filename = file_path+"_reindexed"
destination_path = poscar_filename

with open(destination_path, "w") as poscar_positive_file:
    for item in file_list:
        poscar_positive_file.writelines(item)

In [ ]:
mic_eucledian_distance((0.3447233333218262, 0.6487857534945276, 0.0476361236171942), (0.2153953041113269, 0.8290481479192707, 0.0136611114864024))

In [ ]:
mic_eucledian_distance((0.3447233333218262, 0.6487857534945276, 0.0476361236171942), (0.3035966699081504, 0.5380755899917785, 0.1598068290235508))

In [ ]:
mic_eucledian_distance((0.1686331230974980, 0.5481800698088471, 0.3216526124351740), (0.2153953041113269, 0.8290481479192707, 0.0136611114864024))

In [ ]:
mic_eucledian_distance((0.1686331230974980, 0.5481800698088471, 0.3216526124351740), (0.3035966699081504, 0.5380755899917785, 0.1598068290235508))

In [ ]:
mic_eucledian_distance((0.57837, 0.96250, 0.71012), (0.32196, 0.02615, 0.73041))

In [ ]:
mic_eucledian_distance((0.4306, 0.06943, 0.70497), (0.5694, 0.93057, 0.70497))

In [ ]:
mic_eucledian_distance((0.8127344301972999, 0.0432043026146805, 0.1642113941390748), (0.6539990270722258, 0.0287537941704466, 0.2648261556974179))

In [ ]:
mic_eucledian_distance((0.8127344301972999, 0.0432043026146805, 0.1642113941390748), (0.9665076316391628, 0.1426618873093889, 0.1426618873095025))

In [ ]:
mic_eucledian_distance((0.5220054167499587, 0.1675033740274538, 0.3127842378515202), (0.9665076316391628, 0.1426618873093889, 0.1426618873095025))

In [ ]:
mic_eucledian_distance((0.5220054167499587, 0.1675033740274538, 0.3127842378515202), (0.6539990270722258, 0.0287537941704466, 0.2648261556974179))